In [1]:
###
from typing import List
import json
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, conlist
from typing import List

from promptrefiner import AbstractLLM

In [2]:
###
load_dotenv("env_variables.env")
client = OpenAI()

In [3]:
###
class LlamaCPPModel(AbstractLLM):
    def __init__(self, base_url, api_key):
        super().__init__()
        from openai import OpenAI
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        
    def predict(self, input_text, system_prompt, temperature=0.1, max_tokens = 200):
        response = self.client.chat.completions.create(
        model="mistral",
        messages=[
            {"role": "system", "content": system_prompt},  # Update this as per your needs
            {"role": "user", "content": input_text}
        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )
        llm_response = response.choices[0].message.content
        return llm_response

In [4]:
###
llamamodel = LlamaCPPModel(base_url="http://localhost:8000/v1", api_key="sk-xxx")

In [5]:
from dataclasses import dataclass

@dataclass
class PromptsClass:
    """Class for keeping track of an item in inventory."""
    mistral_system_prompts: List[str]
    target_input: List[str]
    target_output: List[str]
    mistral_responses: List[List[str]]

In [5]:
###
class PromptClass():
    def __init__(self, init_system_prompt: str):
        
        self.target_inputs = []
        self.target_outputs = []

        if not init_system_prompt:
            raise ValueError("init_system_prompt cannot be empty")
        self.llm_system_prompts = [init_system_prompt]

        self.llm_responses = []
    
    def add_system_prompt(self, system_prompt: str) -> None:
        self.llm_system_prompts.append(system_prompt)
    
    def get_last_sys_prompt(self) -> str:
        return self.llm_system_prompts[-1]
    
    def add_evaluation_example(self, input: str, output :str) -> None:
        self.target_inputs.append(input)
        self.target_outputs.append(output)
    
    def add_llm_response(self, response: List[str]) -> None:
        self.llm_responses.append(response)
    
    def generate_and_add_responses(self, llm_model) -> None:
        temp_responses = []
        sys_prompt = self.get_last_sys_prompt()
        for input_text in self.target_inputs:
            output_from_llm = llm_model.predict(input_text=input_text, system_prompt=sys_prompt)
            temp_responses.append(output_from_llm)
        self.add_llm_response(temp_responses)

In [6]:
###
input_evaluation_1 = "She said: 'today was supposed to be a day of celebration and joy in Kansas, instead it is another day where America has experience senselense gun violence' in response to what happened in Kansas near coca-cola branch"

output_evaluation_1 = ["Gun violence", "Coca-cola", "Kansas city"]

input_evaluation_2 = "I would say the best place to go for your honeymoon is Paris, but some say it's overrated"

output_evaluation_2 = ["Paris", "Honeymoon"]

In [7]:
###
init_sys_prompt = """You are an AI designed to find a LIMITED list of GENERAL concepts associated with a given piece of text. """
promptTracker = PromptClass(init_system_prompt = init_sys_prompt)
promptTracker.add_evaluation_example(input_evaluation_1, str(output_evaluation_1))
promptTracker.add_evaluation_example(input_evaluation_2, str(output_evaluation_2))

In [8]:
###
promptTracker.llm_system_prompts

['You are an AI designed to find a LIMITED list of GENERAL concepts associated with a given piece of text. ']

### Creating initial mistral input output

In [9]:
###
for i in range(len(promptTracker.target_inputs)):
    print(promptTracker.target_inputs[i])

She said: 'today was supposed to be a day of celebration and joy in Kansas, instead it is another day where America has experience senselense gun violence' in response to what happened in Kansas near coca-cola branch
I would say the best place to go for your honeymoon is Paris, but some say it's overrated


In [12]:
###
promptTracker.llm_responses

[]

In [13]:
promptTracker.generate_and_add_responses(llamamodel)

### Using OpenAI GPT4 for refinement of system prompt 

In [20]:
###
openai_sys_prompt = """You are an AI assistant who is expert in creating promtps for LLMs. you job is to modify and enhance a prompt for a 7b mistral instruct model. The mistral model is supposed to receive an input text, and return a list of strings, entities, brand names, etc in that input text. The prompt to the mistral model can include some examples that lead the model's behavior.

A number of experiments have been done on different system prompts for mistral and the output. Those experiments which include tested system prompt, tested INPUTs TO MISTRAL, and the resulting outputs from Mistral are provided to you. Your job is to observe the experiments, and come up with a better system prompt for Mistral to achieve the expected output. you can provide some examples, or remove some examples in your suggested system prompt. Remember that total number of examples should be limited to 3, because it adds extra computation and we can't afford it. Note that the examples given in the system prompt of mistral should be enclosed by ### ###. Pay attention to the fact that, you are NOT allowed to use EVALUATION INPUT TO MISTRAL texts in your examples for your suggested mistral system prompt.
"""

In [21]:
promptTracker.llm_responses[0][1]

'1. Paris\n2. Honeymoon\n3. Travel\n4. Overrated\n5. Tourist destination\n6. Romance\n7. Europe\n8. City break\n9. Culture\n10. Sightseeing\n11. Travel experience\n12. Tourism\n13. Holiday\n14. European city\n15. Landmark\n16. Attraction\n17. Eiffel Tower\n18. Louvre Museum\n19. Notre-Dame Cathedral\n20. Seine River\n21. French cuisine\n22. Parisian lifestyle\n23. Romantic getaway\n24. Travel advice\n25. Personal opinion\n26. Tourist opinion\n27. European travel\n28. Parisian landmarks\n29. Parisian attractions\n30. Parisian culture\n31. Parisian cuisine\n3'

In [22]:
def create_openai_user_prompt(prompttracker):
    total_prompt = ""

    for i in range(len(prompttracker.llm_responses)):
        total_prompt += f"""\n\n\n\n
Experiment {i}
Mistral System Prompt:
{prompttracker.llm_system_prompts[i]}
"""
        for j in range(len(prompttracker.target_inputs)):
            total_prompt += f"""\n\n

EVALUATION INPUT {j} TO Local LLM:
{prompttracker.target_inputs[j]}
EVALUATION OUTPUT {j} FROM Local LLM:
{prompttracker.llm_responses[i][j]}
what was EXPECTED to be EVALUATION OUTPUT {j} from Local LLM:
{str(prompttracker.target_outputs[j])} \n\n
"""
        
    return total_prompt

In [23]:
class EnhancedSystemPrompt(BaseModel):
    Enhanced_System_Prompt: str


def request_to_openai(prompttracker):
    openai_user_prompt = create_openai_user_prompt(prompttracker=prompttracker)

    response = client.chat.completions.create(
        temperature = 0.1,
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": openai_sys_prompt},
            {"role": "user", "content": openai_user_prompt},
            ],
        functions=[
            {
            "name": "Enhanced_System_Prompt",
            "description": "Enhanced System Prompt for Local LLM",
            "parameters": EnhancedSystemPrompt.model_json_schema()
            }
        ],
        function_call={"name": "Enhanced_System_Prompt"}
    )
    return json.loads(response.choices[0].message.function_call.arguments)['Enhanced_System_Prompt']

In [25]:
def refine_system_prompt_with_gpt4(number_of_iterations):
    for i in range(number_of_iterations):
        openai_suggestion = request_to_openai(prompttracker=promptTracker)
        promptTracker.add_system_prompt(openai_suggestion)
        

        promptTracker.generate_and_add_responses(llamamodel)

In [26]:
refine_system_prompt_with_gpt4(3)

In [27]:
print(promptTracker.target_outputs)

["['Gun violence', 'Coca-cola', 'Kansas city']", "['Paris', 'Honeymoon']"]


In [29]:
for i in promptTracker.llm_responses:
    print(i)

['1. Gun Violence\n2. Celebration\n3. Joy\n4. Kansas\n5. America\n6. Coca-Cola Branch\n7. Senseless Violence\n8. Shooting Incident\n9. Safety Concerns\n10. Community Event\n11. Tragedy\n12. Public Space\n13. Security\n14. Violence Prevention\n15. Emergency Response\n16. Law Enforcement\n17. Public Awareness\n18. Gun Control\n19. Mourning\n20. Community Support\n', '1. Paris\n2. Honeymoon\n3. Travel\n4. Overrated\n5. Tourist destination\n6. Romance\n7. Europe\n8. City break\n9. Culture\n10. Sightseeing\n11. Travel experience\n12. Tourism\n13. Holiday\n14. European city\n15. Landmark\n16. Attraction\n17. Eiffel Tower\n18. Louvre Museum\n19. Notre-Dame Cathedral\n20. Seine River\n21. French cuisine\n22. Parisian lifestyle\n23. Romantic getaway\n24. Travel advice\n25. Personal opinion\n26. Tourist opinion\n27. European travel\n28. Parisian landmarks\n29. Parisian attractions\n30. Parisian culture\n31. Parisian cuisine\n3']
["['Kansas', 'America'] \n", "['Paris'] \n"]
["['Kansas', 'America'

In [36]:
print(promptTracker.llm_responses[3])

["Output: ['America', 'gun violence'] \n", "['Paris'] \n"]


In [39]:
promptTracker.get_last_sys_prompt()

"You are an AI designed to extract and list a LIMITED number of SPECIFIC entities, brand names, locations, or general concepts directly mentioned or clearly implied in a given piece of text. Your task is to identify and list the explicit entities, brands, locations, or general concepts without inferring or expanding on the concepts. Focus on the most relevant and directly mentioned items in the text.\n\n### Example 1: \nInput: 'The new iPhone 12 is now available at all Apple stores.' \nOutput: ['iPhone 12', 'Apple'] \n\n### Example 2: \nInput: 'The annual marathon in Boston was canceled due to unexpected weather conditions.' \nOutput: ['Boston', 'marathon'] \n\n### Example 3: \nInput: 'A tragic incident of gun violence occurred near the Coca-Cola branch in Kansas City.' \nOutput: ['gun violence', 'Coca-Cola', 'Kansas City'] \n\n"